The program is to find the longest simple (crossing each node exactly once) path in a directed weighted acyclic graph, where weight of an edge is a "distance" beetwen nodes. It uses a generic algorithm provided by pygad library to compute the path.

In [1]:
import networkx as nx
import random
import pygad
import matplotlib.pyplot as plt
import time
import math

Function that generates a graph with given number of nodes. There is a 30% chance that a node will have an edge to another node. Range of weights for each edge is [1, 6]. For testing purposes, the function also returns the longest path in the graph so we can later check if the algorithm found the ideal solution.

In [2]:
def generate_graph(number_of_nodes):
    G = nx.DiGraph()

    # Adding nodes
    G.add_nodes_from(range(number_of_nodes))

    # Adding edges with random weights between 1 and 6
    for i in range(number_of_nodes - 1):
        for j in range(i+1, number_of_nodes):
            if random.random() < 0.3:
                weight = random.randint(1, 6)
                G.add_edge(i, j, weight=weight)
                
    longest_path = nx.dag_longest_path(G, weight='weight')
    
    return (G, longest_path)

Simple function for drawing the graph with weight labels on the edges.

In [3]:
def draw_digraph(graph):
    pos = nx.kamada_kawai_layout(graph)
    nx.draw(graph, pos=pos, with_labels=True)
    labels = nx.get_edge_attributes(graph, 'weight')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
    plt.show()

Normalize solution function is to change the solution from a list of numbers between 0 and 99 which are used to choose indexes of the nodes we want to go to, to the very index of the node. It's used for better readability and because of the way the fitness function is implemented.

In [4]:
def normalize_solution(graph, solution):
    current_node = solution[0] % graph.number_of_nodes()
    normalized_solution = [current_node]
    
    for direction in solution[1:]: # Skip the first node as it is our starting point
        if direction % 2 == 1: # We ignore odd directions, so the algorithm can ignore given genes (the path doesn't have to be the same length as the chromsome).
            break
        out_edges = list(graph.out_edges(current_node))
        if len(out_edges) == 0:
            break
        normalized_direction = (direction // 2) % len(out_edges) # That's the formula with which we normalize the direction. We use modulo so there is always a valid direction to choose.
        # We divide direction by 2 so we can also get the full scope of numbers (0, 1, 2, 3, ...) instead of just (0, 2, 4, 6, ...). That allows the algorithm to choose also odd indices of nodes even tho we ignore odd directions. For example, 8 // 2 = 4 (even result), but 6 // 2 = 3 (odd result).
        normalized_solution.append(normalized_direction)
        current_node = out_edges[normalized_direction][1]
    return normalized_solution

Convert solution to path function is to convert the solution from a list of indexes of the nodes to a list of nodes. It's used for better readability at the end of the program and for the fitness function to punish the solutions that go to the same node more than once.

In [5]:
def convert_solution_to_path(graph, solution):
	normalized_solution = normalize_solution(graph, solution)
	current_node = normalized_solution[0]
	path = [current_node]
	for norm_direction in normalized_solution[1:]:
		out_edges = list(graph.out_edges(current_node))
		current_node = out_edges[norm_direction][1]
		path.append(current_node)
	return path

Firstly, fitness function normalizes the solution (uses another function), so that it represents indexes of nodes instead of a "(random number (0, 99)) % (number of possible nodes to go)". Then, it goes through every element of solution (direction), checks possible edges that we can choose to go through and updates current node with the direction (index) of out edges. It also updates current path length by adding the new edge weight to it. Next, fitness converts the solution to a real path representing numbers of nodes in specific order to go through and checks if any node was visited more than once. If so, it punishes the fitness result harshly, as we only want result paths to be simple. It's worth to mention, that the algorithm selects the starting point itself for better results, because there may be more edges with possibly better paths coming out of node different than, for example the first one in given graph.

In [6]:
def generate_fitness_function(graph):
    def fitness_function(solution, solution_idx):
        normalized_solution = normalize_solution(graph, solution)
        current_node = normalized_solution[0]
        current_path_length = 0
        for normalized_direction in normalized_solution[1:]: # Skip the first node as it is our starting point
            out_edges = list(graph.out_edges(current_node, data=True))
            current_node = out_edges[normalized_direction][1]
            current_path_length += out_edges[normalized_direction][2]['weight']

        path = convert_solution_to_path(graph, solution)
        countByNode = {}
        for node in path:
            if node not in countByNode:
                countByNode[node] = 0
            countByNode[node] += 1
        for node in countByNode:
            countByNode[node] -= 1 # Decrease every node count by 1 so that we don't punish the node for being visited once
        fitness = current_path_length
        for node in countByNode:
            fitness -= 10 * math.pow(countByNode[node], 2) # The more a node is visited, the more we decrease the fitness (square the count to make it more punishing)


        return fitness
    return fitness_function                                            

Initial population is a list of random solutions. Each solution is a list of numbers between 0 and 99, which are used to choose indexes of the nodes we want to go to. The number of solutions is equal to the population size. Note that during algorithm runtime, the solutions values might exceed the range of 0 to 99. There's no statically defined gene space.

In [7]:
# Define the initial population
def create_population(pop_size, nodes_count):
    return [[random.randint(0, 100) for _ in range(nodes_count)] for _ in range(pop_size)]

In [8]:
# Declare the number of nodes in the graph

number_of_nodes = 8

Creating an instance of GA. We use a generate_graph function for each iteration so that every run is different. If the solution for given graph is correct, we measure the time it took to find it and then print an average of all correct solutions times. We're also measuring the accuracy of the algorithm by dividing number of ideal solutions by number of all solutions.

In [9]:
times = []
correct_solutions = 0
iterations = 5
# draw_digraph(graph)
for i in range(iterations):
    subject = generate_graph(number_of_nodes)
    graph = subject[0]
    longest_path = subject[1]
    print("Run ", i+1)
    start = time.time()
    sol_per_pop = 500
    ga = pygad.GA(
                num_generations=700,
                num_parents_mating=10,
                fitness_func=generate_fitness_function(graph),          
                sol_per_pop=sol_per_pop,
                num_genes=len(graph.nodes),
                crossover_type=None, # We don't want to use crossover as it will mess up the path. We fully rely on mutation
                mutation_type="random",
                initial_population=create_population(sol_per_pop, len(graph.nodes)),
                gene_type=int,
                mutation_probability=0.1,
                random_mutation_max_val=3,
                random_mutation_min_val=-3,
                keep_parents=10,
                )


    ga.run()
    end = time.time()
    solution, solution_fitness, _ = ga.best_solution()
    # If the solution is correct, we add the time it took to the list of times and increase the correct solutions counter.
    if list(convert_solution_to_path(graph, solution)) == list(longest_path):
        print("Found ideal solution!")
        correct_solutions += 1
        times.append(end - start)
        

    print("Best solution (path):",  convert_solution_to_path(graph, solution))
    print("Fitness:", solution_fitness)
    print("-------------------------------------------")
    
average_time = 0
for time in times:
    average_time += time
average_time = average_time / len(times)
print("Average time of finding a single ideal solution: ", average_time, " s")
# Accuracy is the percentage of times the algorithm found the ideal solution. We check it with the "nx.dag_longest_path" buit-in function.
accuracy = (correct_solutions / iterations) * 100
print("Accuracy: ", accuracy, " %")

Run  1
Found ideal solution!
Best solution (path): [1, 3, 7]
Fitness: 11.0
-------------------------------------------
Run  2
Found ideal solution!
Best solution (path): [1, 3, 5]
Fitness: 9.0
-------------------------------------------
Run  3
Found ideal solution!
Best solution (path): [0, 2, 3, 5, 6]
Fitness: 14.0
-------------------------------------------
Run  4
Found ideal solution!
Best solution (path): [0, 5, 7]
Fitness: 9.0
-------------------------------------------
Run  5
Found ideal solution!
Best solution (path): [0, 3, 7]
Fitness: 11.0
-------------------------------------------
Average time of finding a single ideal solution:  9.111594486236573  s
Accuracy:  100.0  %


Average time of finding ideal solutions and their percentage in all results for GA (100 iterations):

In [10]:
#   ---------------------------------------------------------------------------------------------------------
#   |   Graph size   |   Generations   |   Sol_per_pop   |   Average time   |   Accuracy   |   Total time   | 
#   |-------------------------------------------------------------------------------------------------------|
#   |   8            |   1000          |   400           |   9.81s          |   94%        |   16m 19s      |
#   |-------------------------------------------------------------------------------------------------------|
#   |   14           |   1500          |   500           |   24.79s         |   83%        |   41m 21s      |
#   |-------------------------------------------------------------------------------------------------------|
#   |   20           |   1500          |   700           |   36.06s         |   63%        |   60m 23s      |
#   ---------------------------------------------------------------------------------------------------------

Bibliography

[1] https://en.wikipedia.org/wiki/Longest_path_problem

[2] https://networkx.org/documentation/stable/reference/classes/digraph.html

[3] https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.dag.dag_longest_path.html

[4] https://networkx.org/documentation/stable/reference/functions.html

[5] https://www.youtube.com/watch?v=YxF-x3imVFA&t=32s